In [38]:
import requests
import numpy as np
import pandas as pd
import sklearn.ensemble as sk
import sklearn
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [39]:
train=pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


1. Creat three train data subsets: (1) with all features (2) without age (3) without age and Fare
===

In [40]:
train1=train[['PassengerId','Pclass','Sex','Age','SibSp','Fare','Survived']].copy()
train2=train[['PassengerId','Pclass','Sex','SibSp','Fare','Survived']].copy()
train3=train[['PassengerId','Pclass','Sex','SibSp','Survived']].copy()
train1=train1.dropna()
train2=train2.dropna()
train3=train3.dropna()
print train1.shape
print train2.shape
print train3.shape
print train.shape

(714, 7)
(891, 6)
(891, 5)
(891, 12)


In [41]:
sex_dic={'male':0,'female':1}
embarked_dic={'S':0,'C':1,'Q':2}
def rename(df):
    df['Sex']=map(lambda x: sex_dic[x],df['Sex'])
    #df['Embarked']=map(lambda x: embarked_dic[x],df['Embarked'])
    return df
train1=rename(train1)
train2=rename(train2)
train3=rename(train3)

In [42]:
sub_train1,sub_val1=sklearn.cross_validation.train_test_split(train1,test_size=0.33,random_state=99)
sub_train1=pd.DataFrame(sub_train1)
sub_train1.columns=train1.columns
sub_val1=pd.DataFrame(sub_val1)
sub_val1.columns=train1.columns
print sub_train1.shape
print sub_val1.shape

sub_train2,sub_val2=sklearn.cross_validation.train_test_split(train2,test_size=0.33,random_state=99)
sub_train2=pd.DataFrame(sub_train2)
sub_train2.columns=train2.columns
sub_val2=pd.DataFrame(sub_val2)
sub_val2.columns=train2.columns
print sub_train2.shape
print sub_val2.shape

sub_train3,sub_val3=sklearn.cross_validation.train_test_split(train3,test_size=0.33,random_state=99)
sub_train3=pd.DataFrame(sub_train3)
sub_train3.columns=train3.columns
sub_val3=pd.DataFrame(sub_val3)
sub_val3.columns=train3.columns
print sub_train3.shape
print sub_val3.shape

(478, 7)
(236, 7)
(596, 6)
(295, 6)
(596, 5)
(295, 5)


2. Import test data and split into three subsets: test1,test2,test3
===

In [43]:
test=pd.read_csv('test.csv')
test_sub=test[['PassengerId','Pclass','Sex','Age','SibSp','Fare']].copy()
print test_sub.shape
test_sub.head()

(418, 6)


,PassengerId,Pclass,Sex,Age,SibSp,Fare
0,892,3,male,34.5,0,7.8292
1,893,3,female,47.0,1,7.0000
2,894,2,male,62.0,0,9.6875
3,895,3,male,27.0,0,8.6625
4,896,3,female,22.0,1,12.2875


In [44]:
test1=test_sub
test1=test1.dropna()
test1=rename(test1)
print test1.shape
test1.head()

(331, 6)


/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PassengerId,Pclass,Sex,Age,SibSp,Fare
0,892,3,0,34.5,0,7.8292
1,893,3,1,47.0,1,7.0000
2,894,2,0,62.0,0,9.6875
3,895,3,0,27.0,0,8.6625
4,896,3,1,22.0,1,12.2875


In [45]:
test_nan=test_sub[test_sub.isnull().any(axis=1)]
print test_nan.shape

test2=test_nan.drop('Age',axis=1)
test2=test2.dropna()
test2=rename(test2)
print test2.shape

test3=test_nan.drop('Age',axis=1)
test3=test3[test3.isnull().any(axis=1)]
test3=rename(test3)
test3=test3.drop('Fare',axis=1)
print test3.shape

print test.shape[0]==test1.shape[0]+test2.shape[0]+test3.shape[0]

(87, 6)
(86, 5)
(1, 4)
True


In [46]:
test3.head()

,PassengerId,Pclass,Sex,SibSp
152,1044,3,0,0


3. Train the train data
===

In [47]:
col=[['Pclass','Sex','Age','SibSp','Fare','intercept'],
      ['Pclass','Sex','SibSp','Fare','intercept'],
        ['Pclass','Sex','SibSp','intercept']]
def logistic_regression_train(train,i,col=col):
    train['intercept']=1.0
    cols=col[i-1]
    logit=sm.Logit(train['Survived'],train[cols])
    result=logit.fit()
    paras=result.params
    return paras

(1) subset 1
---

In [48]:
paras1=logistic_regression_train(sub_train1,1)
paras1

Optimization terminated successfully.
         Current function value: 0.424918
         Iterations 6


Pclass      -1.330308
Sex          2.672016
Age         -0.034210
SibSp       -0.526736
Fare         0.002452
intercept    2.454407
dtype: float64

(2) subset 2
---

In [49]:
paras2=logistic_regression_train(sub_train2,2)
paras2

Optimization terminated successfully.
         Current function value: 0.433616
         Iterations 6


Pclass      -0.907400
Sex          3.029272
SibSp       -0.342919
Fare         0.002864
intercept    0.508612
dtype: float64

(3) subset 3
--

In [50]:
paras3=logistic_regression_train(sub_train3,3)
paras3

Optimization terminated successfully.
         Current function value: 0.434280
         Iterations 6


Pclass      -0.983711
Sex          3.049841
SibSp       -0.318048
intercept    0.741856
dtype: float64

4. Validation and test
===

In [51]:
from math import exp
def probability(row,paras):
    z=paras[len(paras)-1]
    for i in np.arange(0,len(paras)-1):
        z=z+row[i]*paras[i]
    return 1/(1+exp(-1.0*z))

(1) subset 1
----

In [52]:
compare=sub_val1[['Pclass','Sex','Age','SibSp','Fare']].copy()
pred=[]
for ii in np.arange(len(compare.Pclass)):
    pred.append(probability(compare.iloc[ii],paras1))
sub_val1['Pred']=pred

*****
compute confusion matrix with gender considered
*****

In [53]:
temp=sub_val1[['Sex','Pclass','Survived','Pred']].copy()
tempmale=temp[temp['Sex']==0]
tempfemale=temp[temp['Sex']==1]

temp_male_low=tempmale[tempmale['Pclass']<=2]
temp_male_high=tempmale[tempmale['Pclass']>=3]
temp_female_low=tempfemale[tempfemale['Pclass']<=2]
temp_female_high=tempfemale[tempfemale['Pclass']>=3]

In [54]:
print temp_male_low.shape
print temp_male_high.shape
print temp_female_low.shape
print temp_female_high.shape

(66, 4)
(80, 4)
(53, 4)
(37, 4)


In [55]:
def survive_cm(pp,df):
    temp=df[['Survived','Pred']].copy()
    temp['TF']=temp['Pred']>=pp
    cm=sklearn.metrics.confusion_matrix(temp['Survived'],temp['TF'])
    return cm

In [56]:
xlist=np.linspace(0,1,33)
m1=map(lambda x: survive_cm(x,temp_male_low),xlist)
m2=map(lambda x: survive_cm(x,temp_male_high),xlist)
m3=map(lambda x: survive_cm(x,temp_female_low),xlist)
m4=map(lambda x: survive_cm(x,temp_female_high),xlist)

mm=[]
for i1 in m1:
    for i2 in m2:
        for i3 in m3:
            for i4 in m4:
                cm=i1+i2+i3+i4
                mm.append((cm[0,0]+cm[1,1])*1.0/(cm[0,0]+cm[1,1]+cm[0,1]+cm[1,0]))
        
# mmm=np.reshape(mm, (-1, len(xlist)))


# contour_levels = np.linspace(0,1,100)
# plt.contour(xlist,ylist,mmm,levels=contour_levels)
# plt.colorbar()
# plt.show()

In [57]:
print max(mm)
ind=mm.index(max(mm))
print ind

0.792372881356
363726


In [58]:
p4_ind=ind%len(xlist)
p3_ind=ind/len(xlist)%len(xlist)
p2_ind=ind/len(xlist)/len(xlist)%len(xlist)
p1_ind=ind/len(xlist)/len(xlist)/len(xlist)%len(xlist)
print p4_ind,p3_ind,p2_ind,p1_ind
p4=xlist[p4_ind]
p3=xlist[p3_ind]
p2=xlist[p2_ind]
p1=xlist[p1_ind]

0 0 4 10


In [59]:
compare=test1[['Pclass','Sex','Age','SibSp','Fare']].copy()
pred=[]
for ii in np.arange(len(compare.Pclass)):
    pred.append(probability(compare.iloc[ii],paras1))
test1['Pred']=pred
testmale=test1[test1['Sex']==0]
testfemale=test1[test1['Sex']==1]
test_male_low=testmale[testmale['Pclass']<=2]
test_male_high=testmale[testmale['Pclass']>=3]
test_female_low=testfemale[testfemale['Pclass']<=2]
test_female_high=testfemale[testfemale['Pclass']>=3]

test_male_low['Survived']=1*(test_male_low['Pred']>=p1)
test_male_high['Survived']=1*(test_male_high['Pred']>=p2)
test_female_low['Survived']=1*(test_female_low['Pred']>=p3)
test_female_high['Survived']=1*(test_female_high['Pred']>=p4)

test1=(pd.concat([test_male_low,test_male_high,test_female_low,test_female_high])).sort('PassengerId')

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [60]:
test1.head()

,PassengerId,Pclass,Sex,Age,SibSp,Fare,Pred,Survived
0,892,3,0,34.5,0,7.8292,0.063117,0
1,893,3,1,47.0,1,7.0000,0.272506,1
2,894,2,0,62.0,0,9.6875,0.090835,0
3,895,3,0,27.0,0,8.6625,0.080251,0
4,896,3,1,22.0,1,12.2875,0.471599,1


(2) subset 2
---

In [61]:
compare=sub_val2[['Pclass','Sex','SibSp','Fare']].copy()
pred=[]
for ii in np.arange(len(compare.Pclass)):
    pred.append(probability(compare.iloc[ii],paras2))
sub_val2['Pred']=pred

temp=sub_val2[['Sex','Pclass','Survived','Pred']].copy()
tempmale=temp[temp['Sex']==0]
tempfemale=temp[temp['Sex']==1]

temp_male_low=tempmale[tempmale['Pclass']<=2]
temp_male_high=tempmale[tempmale['Pclass']>=3]
temp_female_low=tempfemale[tempfemale['Pclass']<=2]
temp_female_high=tempfemale[tempfemale['Pclass']>=3]

xlist=np.linspace(0,1,33)
m1=map(lambda x: survive_cm(x,temp_male_low),xlist)
m2=map(lambda x: survive_cm(x,temp_male_high),xlist)
m3=map(lambda x: survive_cm(x,temp_female_low),xlist)
m4=map(lambda x: survive_cm(x,temp_female_high),xlist)

mm=[]
for i1 in m1:
    for i2 in m2:
        for i3 in m3:
            for i4 in m4:
                cm=i1+i2+i3+i4
                mm.append((cm[0,0]+cm[1,1])*1.0/(cm[0,0]+cm[1,1]+cm[0,1]+cm[1,0]))

print max(mm)
ind=mm.index(max(mm))
print ind

p4_ind=ind%len(xlist)
p3_ind=ind/len(xlist)%len(xlist)
p2_ind=ind/len(xlist)/len(xlist)%len(xlist)
p1_ind=ind/len(xlist)/len(xlist)/len(xlist)%len(xlist)
print p4_ind,p3_ind,p2_ind,p1_ind
p4=xlist[p4_ind]
p3=xlist[p3_ind]
p2=xlist[p2_ind]
p1=xlist[p1_ind]

compare=test2[['Pclass','Sex','SibSp','Fare']].copy()
pred=[]
for ii in np.arange(len(compare.Pclass)):
    pred.append(probability(compare.iloc[ii],paras2))
test2['Pred']=pred
testmale=test2[test2['Sex']==0]
testfemale=test2[test2['Sex']==1]
test_male_low=testmale[testmale['Pclass']<=2]
test_male_high=testmale[testmale['Pclass']>=3]
test_female_low=testfemale[testfemale['Pclass']<=2]
test_female_high=testfemale[testfemale['Pclass']>=3]

test_male_low['Survived']=1*(test_male_low['Pred']>=p1)
test_male_high['Survived']=1*(test_male_high['Pred']>=p2)
test_female_low['Survived']=1*(test_female_low['Pred']>=p3)
test_female_high['Survived']=1*(test_female_high['Pred']>=p4)

test2=(pd.concat([test_male_low,test_male_high,test_female_low,test_female_high])).sort('PassengerId')

0.806779661017
687182

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing


23 0 4 19


In [62]:
test2.head()

,PassengerId,Pclass,Sex,SibSp,Fare,Pred,Survived
10,902,3,0,0,7.8958,0.100564,0
22,914,1,1,0,31.6833,0.938267,1
29,921,3,0,2,21.6792,0.055339,0
33,925,3,1,1,23.4500,0.631794,0
36,928,3,1,0,8.0500,0.698199,0


(3) subset 3
---

In [63]:
compare=sub_val3[['Pclass','Sex','SibSp']].copy()
pred=[]
for ii in np.arange(len(compare.Pclass)):
    pred.append(probability(compare.iloc[ii],paras3))
sub_val3['Pred']=pred

temp=sub_val3[['Sex','Pclass','Survived','Pred']].copy()
tempmale=temp[temp['Sex']==0]
tempfemale=temp[temp['Sex']==1]

temp_male_low=tempmale[tempmale['Pclass']<=2]
temp_male_high=tempmale[tempmale['Pclass']>=3]
temp_female_low=tempfemale[tempfemale['Pclass']<=2]
temp_female_high=tempfemale[tempfemale['Pclass']>=3]

xlist=np.linspace(0,1,33)
m1=map(lambda x: survive_cm(x,temp_male_low),xlist)
m2=map(lambda x: survive_cm(x,temp_male_high),xlist)
m3=map(lambda x: survive_cm(x,temp_female_low),xlist)
m4=map(lambda x: survive_cm(x,temp_female_high),xlist)

mm=[]
for i1 in m1:
    for i2 in m2:
        for i3 in m3:
            for i4 in m4:
                cm=i1+i2+i3+i4
                mm.append((cm[0,0]+cm[1,1])*1.0/(cm[0,0]+cm[1,1]+cm[0,1]+cm[1,0]))

print max(mm)
ind=mm.index(max(mm))
print ind

p4_ind=ind%len(xlist)
p3_ind=ind/len(xlist)%len(xlist)
p2_ind=ind/len(xlist)/len(xlist)%len(xlist)
p1_ind=ind/len(xlist)/len(xlist)/len(xlist)%len(xlist)
print p4_ind,p3_ind,p2_ind,p1_ind
p4=xlist[p4_ind]
p3=xlist[p3_ind]
p2=xlist[p2_ind]
p1=xlist[p1_ind]

compare=test3[['Pclass','Sex','SibSp']].copy()
pred=[]
for ii in np.arange(len(compare.Pclass)):
    pred.append(probability(compare.iloc[ii],paras3))
test3['Pred']=pred
testmale=test3[test3['Sex']==0]
testfemale=test3[test3['Sex']==1]
test_male_low=testmale[testmale['Pclass']<=2]
test_male_high=testmale[testmale['Pclass']>=3]
test_female_low=testfemale[testfemale['Pclass']<=2]
test_female_high=testfemale[testfemale['Pclass']>=3]

test_male_low['Survived']=1*(test_male_low['Pred']>=p1)
test_male_high['Survived']=1*(test_male_high['Pred']>=p2)
test_female_low['Survived']=1*(test_female_low['Pred']>=p3)
test_female_high['Survived']=1*(test_female_high['Pred']>=p4)

test3=(pd.concat([test_male_low,test_male_high,test_female_low,test_female_high])).sort('PassengerId')

0.803389830508
543434
23 0 4 15


In [36]:
matrix=np.asmatrix(mmm)
#print matrix.argmax()
#print matrix.argmax(0)
#print matrix.argmax(1)
print np.max(matrix)
ppm=xlist[440]
ppf=xlist[698]

0.803389830508


In [64]:
test3.head()

,PassengerId,Pclass,Sex,SibSp,Pred,Survived
152,1044,3,0,0,0.098921,0


5.Results
===

In [65]:
result1=test1[['PassengerId','Survived']].copy()
result2=test2[['PassengerId','Survived']].copy()
result3=test3[['PassengerId','Survived']].copy()

In [66]:
frames=[result1,result2,result3]
results=pd.concat(frames)

In [67]:
results.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [68]:
results.to_csv('titanic_logistic_gender_pclass_drop.csv',header=True,index=False)

In [ ]:
ht